In [12]:
import numpy as np
import pandas as pd

In [15]:
data = pd.read_excel("../data/spx_returns_weekly.xlsx", sheet_name= "s&p500 rets")
data.set_index('date', inplace = True)
Tickers = ['AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AVGO', 'BRK/B', 'LLY']
df = data[Tickers].copy()
ETF_data = pd.read_excel("../data/spx_returns_weekly.xlsx", sheet_name= "benchmark rets")
ETF_data.set_index('date', inplace = True)
TICK_ETF = 'SPY'
df[TICK_ETF] = ETF_data[TICK_ETF].copy()
df.head()

,AAPL,NVDA,MSFT,GOOGL,AMZN,META,TSLA,AVGO,BRK/B,LLY,SPY
date,,,,,,,,,,,
2015-01-09,0.024514,-0.009315,0.009195,-0.054445,-0.037534,-0.009055,-0.057685,0.047971,0.002011,-0.001855,-0.005744
2015-01-16,-0.053745,0.000836,-0.020131,0.019448,-0.020880,-0.032931,-0.065760,-0.010268,-0.001739,0.010726,-0.012827
2015-01-23,0.065950,0.037578,0.020329,0.061685,0.074431,0.035255,0.042575,0.030500,-0.000603,0.020514,0.016565
2015-01-30,0.036997,-0.072636,-0.143706,-0.008130,0.134900,-0.024669,0.011476,-0.038331,-0.034938,-0.001802,-0.026931
2015-02-06,0.019114,0.062269,0.049753,-0.006812,0.055737,-0.018967,0.067589,0.018037,0.043569,-0.022778,0.030584


In [16]:
#1.1 
#Report the weights of the constrained portfolio.
#Report the mean, volatility, and Sharpe ratio of the resulting portfolio.

from scipy.optimize import minimize

TARGET_MEAN = 0.2 #annualized mean
FREQ = 52
cov = df.cov() *FREQ
mean = df.mean() *FREQ

def objective(w):        
    return (w.T @ cov @ w)

def fun_constraint_capital(w):
     return np.sum(w) - 1

def fun_constraint_mean(w):
     return (mean @ w) - TARGET_MEAN

bound = tuple((-0.2, 0.35) for _ in range(len(Tickers)+1))

constraint_capital = {'type': 'eq', 'fun': fun_constraint_capital}
constraint_mean = {'type': 'eq', 'fun': fun_constraint_mean}
constraints = ([constraint_capital, constraint_mean])

w0 = np.ones(len(Tickers)+1)/ (len(Tickers)+1)

result = minimize(objective, w0, method='SLSQP',bounds= bound, constraints=constraints )

w = result.x
mv = w.T @ mean
vol = w.T @ cov @ w
sd = np.sqrt(vol)
Sharp = mv/sd
print("Profolio weight: ", w, ", Mean_return: ", mv, ", Volatility: ", vol, "Sharp Ratio: ", Sharp)



Profolio weight:  [ 0.02924139 -0.01434355  0.14450838  0.00906349  0.09385808  0.00287445
 -0.01507129  0.03624973  0.35        0.21308467  0.15053463] , Mean_return:  0.1999999999982956 , Volatility:  0.027188431579310227 Sharp Ratio:  1.2129360892747836


In [ ]:
# 1.2
#Compare these weights to the assets’ Sharpe ratios and means.
#Do the most extreme positions also have the most extreme Sharpe ratios and means?
#Why?

Table = pd.DataFrame(w, index=Tickers + [TICK_ETF], columns= ['Profolio Weight'])

Table['Mean Return'] = mean
Table['Sharp Ratios'] = mean/ (df.std() * np.sqrt(FREQ))
Table.sort_values(by= 'Profolio Weight', ascending= False, inplace= True)
Table

# A: We can clearly see by Table that the most extreme positions do not have the most extreme Sharpe ratios and means.
# A: This is because the weight of MV optimize method values an assets low correlation with other assets than its individual Sharp Ratio.

,Profolio Weight,Mean Return,Sharp Ratios
BRK/B,0.350000,0.135025,0.708223
LLY,0.213085,0.281542,0.994919
SPY,0.150535,0.131264,0.768182
MSFT,0.144508,0.261402,1.089266
AMZN,0.093858,0.293447,0.958976
AVGO,0.036250,0.394854,1.052574
AAPL,0.029241,0.238714,0.862938
GOOGL,0.009063,0.216800,0.774698
META,0.002874,0.261924,0.745509
NVDA,-0.014344,0.645580,1.393490


In [ ]:
# 1.3
#Compare the bounded portfolio weights to the unbounded portfolio weights (obtained from optimizing without the inequality constraints, keeping the equality constraints.)

"""
TARGET_MEAN = 0.2 #annualized mean
FREQ = 52
cov = df.cov() *FREQ
mean = df.mean() *FREQ


def objective(w):        
    return (w.T @ cov @ w)

def fun_constraint_capital(w):
     return np.sum(w) - 1

def fun_constraint_mean(w):
     return (mean @ w) - TARGET_MEAN

bound = tuple((-0.2, 0.35) for _ in range(len(Tickers)+1))

constraint_capital = {'type': 'eq', 'fun': fun_constraint_capital}
constraint_mean = {'type': 'eq', 'fun': fun_constraint_mean}
constraints = ([constraint_capital, constraint_mean])

w0 = np.ones(len(Tickers)+1)/ (len(Tickers)+1)
"""

result_un = minimize(objective, w0, method='SLSQP', constraints=constraints )

w_un = result_un.x
mv_un = w_un.T @ mean
vol_un = w_un.T @ cov @ w_un
sd_un = np.sqrt(vol_un)
Sharp_un = mv_un/sd_un
Con_dic = {"Mean_return: ": mv, "Volatility: ": vol, "Sharp Ratio: ": Sharp}
Un_dic = {"Mean_return: ": mv_un, "Volatility: ": vol_un, "Sharp Ratio: ": Sharp_un}
Compare_Table = pd.DataFrame.from_dict({'Constrained': Con_dic, 'Unconstrained':Un_dic}, orient= 'columns')
Compare_Table



,Constrained,Unconstrained
Mean_return:,0.200000,0.200000
Volatility:,0.027188,0.027181
Sharp Ratio:,1.212936,1.213103
